In [2]:
# dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine

import datetime

# postgres pull

db_password = "postgres"
db_user = "postgres"
db_name = "Youtube P3"
endpoint = 'youtube.cb1bticre0py.us-east-1.rds.amazonaws.com'

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

og_data = pd.read_sql('SELECT * FROM all_data;', con = engine)

In [3]:

slimmer_unique=pd.read_csv('cleaned.csv')  



In [4]:
# X = slim_unique.drop(columns = 'target').values

# y = slim_unique['target'].values
X = slimmer_unique.filter(['category','publish_to_trend','likes_ratio','comments_ratio',"views","likes","dislikes", "comments"], axis=1)
# X = slimmer_unique.drop(columns = ['target', 'trend_days']).values


y = slimmer_unique['trend_days'].values

print(X.shape)
print(y.shape)



(241112, 8)
(241112,)


In [5]:
y

array([4., 1., 1., ..., 1., 1., 1.])

In [6]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
import sklearn as skl

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create scaler instance
# X_scaler = skl.preprocessing.StandardScaler()


In [25]:

# Fit the scaler
# X_scaler.fit(X_train)


In [26]:

# Scale the data
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)




In [7]:
import tensorflow as tf

In [28]:
# Create a Keras Sequential model and add more than one Dense hidden layer
# nn_model = tf.keras.models.Sequential()

# nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=8))

# nn_model.add(tf.keras.layers.Dense(units=10, activation="relu"))

# nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# # Check the structure of the Sequential model
# nn_model.summary()

In [29]:
## Create a method that creates a new Sequential model with hyperparameter options
# def create_model(hp):
#     nn_model = tf.keras.models.Sequential()

#     # Allow kerastuner to decide which activation function to use in hidden layers
#     activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
#     # Allow kerastuner to decide number of neurons in first layer
#     nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
#         min_value=1,
#         max_value=4,
#         step=2), activation=activation, input_dim=3))

#     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#     for i in range(hp.Int('num_layers', 1, 4)):
#         nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
#             min_value=1,
#             max_value=4,
#             step=2),
#             activation=activation))
    
#     nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#     # Compile the model
#     nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
#     return nn_model

In [30]:
# # Import the kerastuner library
# import keras_tuner as kt

# tuner = kt.Hyperband(
#     create_model,
#     objective="val_accuracy",
#     max_epochs=20,
#     hyperband_iterations=2)
# # Run the kerastuner search for best hyperparameters
# tuner.search(X_train,y_train,epochs=20,validation_data=(X_test,y_test))
# # Get best model hyperparameters
# best_hyper = tuner.get_best_hyperparameters(1)[0]
# best_hyper.values
# # Evaluate best model against full test data
# best_model = tuner.get_best_models(1)[0]
# model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [8]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
#need to update

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Compile the model and train over more than 100 epochs
# nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# fit_model = nn_model.fit(X_train, y_train, epochs=200)

In [33]:
# building a model using a model builder function
def model_builder(hp):
    """
    Build model for hyperparameters tuning
    
    hp: HyperParameters class instance
    """
    
    # defining a set of hyperparametrs for tuning and a range of values for each
    start_neurons = hp.Int(name = 'start_neurons', min_value = 16, max_value = 128, step = 16)
    net_depth = hp.Int(name = 'net_depth', min_value = 2, max_value = 6)
    dropout = hp.Boolean(name = 'dropout', default = False)
    bn_after_act = hp.Boolean(name = 'bn_after_act', default = False)
    activation = hp.Choice(name = 'activation', values = ['mish', 'elu', 'lrelu'], ordered = False)
    
    input_size = (544,544,3)
    target_labels = [str(i) for i in range(21)]
    
    # building a model
    model = unet(input_size = input_size,
              start_neurons = start_neurons,
              net_depth = net_depth,
              output_classes = len(target_labels),
              dropout = dropout,
              bn_after_act = bn_after_act,
              activation = activation)
    
    # model compilation
    model.compile(optimizer = Adam(lr = 1e-3),
                  loss = weighted_cross_entropy,
                  metrics = [f1, precision, recall, iou])
    
    return model
import keras_tuner
    # tuner initialization
tuner = keras_tuner.Hyperband(hypermodel = model_builder,
                     objective = keras_tuner.Objective("val_f1", direction="max"),
                     max_epochs = 20,
					 project_name='hyperband_tuner')

tuner.search(training_data=train_dg,
             steps_per_epoch=batches_per_epoch,
             validation_data=valid_dg,
             validation_steps=len(glob(img_dir + '/*')) / valid_batch_size,
             epochs=50,
             shuffle=True,
             verbose=1,
             initial_epoch=0,
             callbacks=[ClearTrainingOutput()],
             use_multiprocessing=True,
             workers=6)

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

NameError: name 'u' is not defined

In [ ]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset.
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# # dfTest[['A', 'B']] = scaler.fit_transform(dfTest[['A','B']].to_numpy())

# slim_unique[
#             [
#             'trending_days',
#             'view_count', 
#             'views_while_trending',
#             'pretrend_view_count_temp', 
#             'while_trend_views_count_temp',
#             'likes_count', 
#             'likes_while_trending', 
#             'pretrend_likes_count_temp',
#             'while_trend_likes_count_temp',
#             'dislikes_count',
#             'dislikes_while_trending', 
#             'pretrend_dislikes_count_temp',
#             'while_trend_dislikes_count_temp', 
#             'comment_count',
#             'comments_while_trending', 
#             'pretrend_comment_count_temp',
#             'while_trend_comment_count_temp'
#             ]
#            ] = scaler.fit_transform(slim_unique[
#                                                 [
#                                                 'trending_days',
#                                                 'view_count', 
#                                                 'views_while_trending',
#                                                 'pretrend_view_count_temp', 
#                                                 'while_trend_views_count_temp',
#                                                 'likes_count', 
#                                                 'likes_while_trending', 
#                                                 'pretrend_likes_count_temp',
#                                                 'while_trend_likes_count_temp',
#                                                 'dislikes_count',
#                                                 'dislikes_while_trending', 
#                                                 'pretrend_dislikes_count_temp',
#                                                 'while_trend_dislikes_count_temp', 
#                                                 'comment_count',
#                                                 'comments_while_trending', 
#                                                 'pretrend_comment_count_temp',
#                                                 'while_trend_comment_count_temp'
#                                                 ]
#                                                ]
#                                    )
# slim_unique

In [ ]:
def test_model(model, data):
    X_train, X_test, y_train, y_test = data
    reg = model.fit(X_train, y_train)
    print(f'Model: {type(reg).__name__}')
    print(f'Train score: {reg.score(X_train, y_train)}')
    print(f'Test Score: {reg.score(X_test, y_test)}\n')
    plt.show()   

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import SVR

In [ ]:
data = [X_train, X_test, y_train, y_test]

In [ ]:
unique = unique.reset_index()

In [ ]:
test_model(LinearRegression(), data)
test_model(Lasso(max_iter=2000),data)
test_model(KNeighborsRegressor(), data)
test_model(RandomForestRegressor(), data)
test_model(ExtraTreesRegressor(), data)
test_model(AdaBoostRegressor(), data)
test_model(SVR(C=1.0, epsilon=0.2), data)